In [1]:
import numpy as np
import pandas as pd

In [2]:
dataset = pd.read_csv(r'C:\Users\Admin\Downloads\Complaint.csv')
dataset.head()

,Complaint
0,Comcast Cable Internet Speeds
1,Payment Disappear - Service Got Disconnected
2,Speed And Service
3,Comcast Imposed A New Usage Cap Of 300Gb That ...
4,Comcast Not Working And No Service To Boot


In [3]:
import re
import nltk

nltk.download('stopwords')

from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()

all_stopwords = stopwords.words('english')
all_stopwords.remove('not')

[nltk_data] Error loading stopwords: <urlopen error [Errno 11001]
[nltk_data]     getaddrinfo failed>


In [4]:
corpus=[]

for i in range(0, 2223):
    review = re.sub('[^a-zA-Z]', ' ', dataset['Complaint'][i])
    review = review.lower()
    review = review.split()
    review = [ps.stem(word) for word in review if not word in set(all_stopwords)]
    review = ' '.join(review)
    corpus.append(review)

In [5]:
from sklearn.feature_extraction.text import CountVectorizer
import pickle
# Now we have a pickled trained model, we can use this for our flask app!
with open(r'C:\Users\Admin\Downloads\c1_BoW_Sentiment_Model.pkl', 'rb') as to_read:
       cv= pickle.load(to_read)

In [6]:
X_fresh = cv.transform(corpus).toarray()
X_fresh.shape

(2223, 1097)

In [7]:
with open(r'C:\Users\Admin\Downloads\finalized.pickle', 'rb') as to_read:
    classifier= pickle.load(to_read)

In [8]:
y_pred = classifier.predict(X_fresh)
print(y_pred)

[3 0 3 ... 2 4 3]


In [9]:
dataset['predicted_label'] = y_pred.tolist()
dataset.head()

,Complaint,predicted_label
0,Comcast Cable Internet Speeds,3
1,Payment Disappear - Service Got Disconnected,0
2,Speed And Service,3
3,Comcast Imposed A New Usage Cap Of 300Gb That ...,4
4,Comcast Not Working And No Service To Boot,4


In [27]:
new_review = 'Comcast Not Working And No Service To Boot'
new_review = re.sub('[^a-zA-Z]', ' ', new_review)
new_review = new_review.lower()
new_review = new_review.split()
ps = PorterStemmer()
all_stopwords = stopwords.words('english')
all_stopwords.remove('not')
new_review = [ps.stem(word) for word in new_review if not word in set(all_stopwords)]
new_review = ' '.join(new_review)
new_corpus = [new_review]
new_X_test = cv.transform(new_corpus).toarray()
new_y_pred = classifier.predict(new_X_test)
print(new_y_pred)
if new_y_pred==0:
    print('Billing')
elif new_y_pred==1:
    print('Email')
elif new_y_pred==2:
    print('Grievance')
elif new_y_pred==3:
    print('Internet')
elif new_y_pred==4:
    print('Network')
elif new_y_pred==5:
    print('Other')
else:
     print('Outages')

[4]
Network
